In [3]:
from glob import glob 
from tqdm import tqdm
from PIL import Image, ImageChops, ImageEnhance
import os 

In [4]:
total_files = glob("../input/casia-dataset/CASIA2/**/*")
print('total files', len(total_files))

# types of files
types = set()
for file in total_files:
    types.add(file.split(".")[-1])
print('types of files in the folder', types)

# jpg files
jpg_files = glob("../input/casia-dataset/CASIA2/*/*.jpg")
print('jpg files: ', len(jpg_files))

#tif files
tif_files = glob("../input/casia-dataset/CASIA2/**/*.tif")
print('tif files: ', len(tif_files))

#TOTAL jpg file whose name starts with Tp
tp_jpg_files = glob("../input/casia-dataset/CASIA2/**/Tp*")
print('tp_jpg_files: ', len(tp_jpg_files))
print('first 5 tp_jpg_files: ', tp_jpg_files[:5])

total files 0
types of files in the folder set()
jpg files:  0
tif files:  0
tp_jpg_files:  0
first 5 tp_jpg_files:  []


In [5]:
def convert_to_ela_image(image_path, quality=90):
    # Save the image at the given quality
    temp_file = 'temp.jpg'
    im = Image.open(image_path).convert('RGB')
    im.save(temp_file, 'JPEG', quality=quality)

    # Open the saved image and the original image
    saved = Image.open(temp_file)
    orignal = Image.open(image_path)

    # Find the absolute difference between the images
    diff = ImageChops.difference(orignal, saved)

    # Normalize the difference by multiplying with a scale factor and convert to grayscale
    extrema = diff.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    scale = 255.0 / max_diff
    diff = ImageEnhance.Brightness(diff).enhance(scale)

    # Remove the temporary file
    os.remove(temp_file)

    return diff

In [6]:
images = jpg_files
print('jpeg files', len(images))
images.extend(tif_files)
print('total images', len(images))

jpeg files 0
total images 0


In [7]:
# find the number of files whose name starts with Tp in a list 
def find_tp_files(files):
    tp_files, au_files = 0, 0
    for file in files:
        if file.split("/")[-1].split("_")[0] == 'Tp':
            tp_files += 1
        if file.split("/")[-1].split("_")[0] == "Au":
            au_files += 1
    return tp_files, au_files
find_tp_files(images)

(0, 0)

In [8]:
for image in tqdm(images[:11957]):
    # converting image to ela image
    ela_image = convert_to_ela_image(image)
    # creating a new directory for ela images
    if not os.path.exists('/CASIA2_ELA'):
        os.mkdir('/CASIA2_ELA')
#     print('/CASIA2_ELAtp/' + image.split('/')[-1])
    ela_image.save('/CASIA2_ELA/' + image.split('/')[-1])

0it [00:00, ?it/s]


In [9]:
# types of files
total_files = glob('/CASIA2_ELA/*')
print('total els files', len(total_files))
types = set()
for file in total_files:
    types.add(file.split(".")[-1])
print('types of files in the folder', types)

total els files 0
types of files in the folder set()


In [10]:
find_tp_files(total_files)

(0, 0)

In [11]:
# counting the number of each class in the dataset
def count_classes(dataloader):
    tempered, original = 0, 0
    for images, labels in tqdm(dataloader):
        for label in labels:
            if label == 1:
                tempered += 1
            else:
                original += 1
    print(f"Tempered: {tempered}, Original: {original}")